### 9.4.1 ランダムフォレスト

In [ ]:
library(caret)
library(tidyverse)
my_data <- iris

my_model <- train(form = Species ~ ., data = my_data, method = "rf",
                  tuneGrid = data.frame(mtry = 2:4), # 省略可
                  trControl = trainControl(method = "LOOCV"))
my_model$results
#>   mtry Accuracy Kappa
#> 1    2     0.96  0.94
#> 2    3     0.96  0.94
#> 3    4     0.96  0.94

### 9.4.2 ブースティング

In [ ]:
library(doParallel)
cl <- makeCluster(detectCores())
registerDoParallel(cl)

my_model <- train(
  form = Species ~ ., data = my_data, method = "xgbTree",
  tuneGrid = expand.grid(                     # ここから
    nrounds          = c(50, 100, 150),
    max_depth        = c(1, 2, 3),
    eta              = c(0.3, 0.4),
    gamma            = 0,
    colsample_bytree = c(0.6, 0.8),
    min_child_weight = 1,
    subsample        = c(0.5, 0.75, 1)),
  trControl = trainControl(method = "LOOCV")) # ここまで省略可
my_model$results %>% filter(Accuracy == max(Accuracy)) %>% t
#>                        [,1]       [,2]
#> nrounds          50.0000000 50.0000000
#> max_depth         1.0000000  2.0000000
#> eta               0.3000000  0.4000000
#> gamma             0.0000000  0.0000000
#> colsample_bytree  0.8000000  0.8000000
#> min_child_weight  1.0000000  1.0000000
#> subsample         0.5000000  0.7500000
#> Accuracy          0.9533333  0.9533333
#> Kappa             0.9300000  0.9300000

### 9.4.3 変数の重要度

In [ ]:
my_model <- train(form = Species ~ ., data = my_data, method = "rf")
ggplot(varImp(my_model))